In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Mar  3 05:13:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
import ee
import rasterio
import numpy as np
import os
import pickle
import georasters as gr

In [7]:
# import ee

# # Authenticate and initialize Earth Engine
# ee.Authenticate()
# ee.Initialize(project="ee-clouderahackathon")

# county_ids = [
#     1, 3, 5, 7, 9, 11, 13, 15, 17, 19,
#     21, 23, 25, 27, 29, 31, 33, 35, 37, 39,
#     41, 43, 45, 47, 49, 51, 53, 55, 57, 59,
#     61, 63, 65, 67, 69, 71, 73, 75, 77, 79,
#     81, 83, 85, 87, 89, 91, 93, 95, 97, 99,
#     101, 103, 105
# ]

# # Define the region of interest (North Dakota bounding box)
# region = ee.FeatureCollection('TIGER/2018/States').filter(ee.Filter.eq('NAME', 'North Dakota')).geometry()

# # Load MODIS SR data for the desired date range and select SR band
# modis_sr = ee.ImageCollection('MODIS/061/MOD09CMG') \
#              .filterDate('2011-04-01', '2020-09-30') \
#              .select(['Coarse_Resolution_Surface_Reflectance_Band_1',
#       'Coarse_Resolution_Surface_Reflectance_Band_2',
#       'Coarse_Resolution_Surface_Reflectance_Band_3',
#         'Coarse_Resolution_Surface_Reflectance_Band_4',
#       'Coarse_Resolution_Surface_Reflectance_Band_5',
#       'Coarse_Resolution_Surface_Reflectance_Band_6',
#         'Coarse_Resolution_Surface_Reflectance_Band_7'])

# # Load the counties of North Dakota
# counties_sr = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', '38'))

# # Define a function to parse county feature properties
# def parse_county(feature):
#     return feature.set('COUNTYFP', ee.Number.parse(feature.get('COUNTYFP')))

# # Map the function over the counties
# counties_nd = counties_sr.map(parse_county)

# # Define the export function
# def export_sr_image_to_drive(image, export_name):
#     task = ee.batch.Export.image.toDrive(image=image.toFloat(),
#                                          description=export_name,
#                                          folder='GEE_Exports',
#                                          fileNamePrefix=export_name,
#                                          region=region,
#                                          scale=250,
#                                          crs='EPSG:4326')
#     task.start()

# # Iterate over each year
# for year in range(2011, 2021):
#     # Filter sr images for the current year
#     modis_sr_year = modis_sr.filterDate(str(year) + '-04-01', str(year) + '-09-30')

#     # Calculate mean sr for the current year
#     mean_sr_year = modis_sr_year.mean()

#     # Iterate over each county
#     for county_id in county_ids:  # North Dakota has 53 counties
#         # Filter the county
#         county = counties_nd.filterMetadata('COUNTYFP', 'equals', county_id).first()

#         # Clip the SR image to the county
#         clipped_sr = mean_sr_year.clip(county.geometry())

#         # Export the clipped SR image to Google Drive
#         export_name = f'SR_NorthDakota_County_{county_id}_Year_{year}'
#         export_sr_image_to_drive(clipped_sr, export_name)

# print('Batch exporting SR images for North Dakota counties from April to September for the past 10 years.')

In [8]:
def tif_preprocessing(img_path):
    # Open GeoTIFF file
    with rasterio.open(img_path) as src:
        # Print raster properties
        #print("Raster bounds:", src.bounds)
        #print("Raster shape:", src.shape)

        # Read raster data
        raster_data = src.read()  # Read the first band
        raster_data = np.array(raster_data)  # Convert to NumPy array
        #cupy_raster_data = cp.asarray(raster_data)
        # Print raster data
        #print("Raster data:", raster_data)
        return raster_data

In [10]:
dir_path = "/content/drive/MyDrive/Conservatives-Official/"
img_paths = os.listdir('/SurfaceReflectance/Data')
print("No of Tif files:", len(img_paths))

No of Tif files: 530


In [11]:
nd_imgs = []
for path in img_paths:
    parts = path.split("_")
    if parts[1] == "NorthDakota":
        year = int(parts[-1].split(".")[0])  # Extract year from the filename
        if 2011 <= year <= 2020:
            nd_imgs.append(path)
    else:
        continue

print(len(nd_imgs))


530


In [12]:
# for path in img_paths:
#       with rasterio.open(dir_path + path) as src:
#         print(path)
#         # Print raster properties
#         #print("Raster bounds:", src.bounds)
#         #print("Raster shape:", src.shape)

#         # Read raster data
#         raster_data = src.read()  # Read the first band
#         raster_data = np.array(raster_data)  # Convert to NumPy array
#         #cupy_raster_data = cp.asarray(raster_data)
#         # Print raster data
#         #print("Raster data:", raster_data)
#         print(raster_data)

In [13]:
def replace_nan_inf(tif_arr):
  # Iterate over each image in the array
  for i in range(tif_arr.shape[0]):
    # Replace NaN values with 0 and ensure data type is float
    tif_arr[i] = np.nan_to_num(tif_arr[i], nan=0, posinf=0, neginf=0).astype(float)
  return tif_arr

def remove_sparsity(image):
    # Find non-zero pixels
    non_zero_pixels = np.nonzero(image)

    # Calculate bounding box with margin
    min_y = max(0, np.min(non_zero_pixels[0]) - 5)
    max_y = min(image.shape[0], np.max(non_zero_pixels[0]) + 6)
    min_x = max(0, np.min(non_zero_pixels[1]) - 5)
    max_x = min(image.shape[1], np.max(non_zero_pixels[1]) + 6)

    # Crop the image to the bounding box
    cropped_image = image[min_y:max_y, min_x:max_x]

    return cropped_image

In [14]:
def process_tif_images(img_paths, dir_path):
    # Calculate the total number of images
    total_images = len(img_paths)

    # Initialize lists for cropped images, shapes, county IDs, and years
    cropped_images = []
    cropped_img_shapes = []
    tif_county_id = []
    tif_county_year = []

    # Find the maximum height and width among all images
    max_height, max_width = 0, 0
    for img_path in img_paths:
        # Read TIFF image and remove sparsity
        tif_img = tif_preprocessing(dir_path + img_path)
        print("file_path:", img_path)
        cropped_image = remove_sparsity(replace_nan_inf(tif_img))
        print("Image cropped")
        print("cropped_image shape: ",cropped_image.shape)

        # Append cropped image and its shape
        cropped_images.append(cropped_image)
        cropped_img_shapes.append(cropped_image.shape)
        print("Image Appended to cropped_images")

        # Extract county ID and year from the image path
        county_id = int(img_path.split("_")[3])
        year = int(img_path.split("_")[-1][:-4])

        # Append county ID and year
        tif_county_id.append(county_id)
        print("county_id = ",county_id)
        tif_county_year.append(year)
        print("year = ",year)

        # Update maximum height and width
        max_height = max(max_height, cropped_image.shape[1])
        max_width = max(max_width, cropped_image.shape[2])
        print("dimensions: ",max_height,max_width)

    # Pad each image to match the maximum height and width
    padded_images = []
    print("padding started")
    for image, shape in zip(cropped_images, cropped_img_shapes):
        print("Inside padding for loop")
        pad_height = max_height - shape[1]
        pad_width = max_width - shape[2]
        padded_image = np.pad(image, ((0, 0),(0, pad_height), (0, pad_width)), mode='constant')
        padded_images.append(padded_image)
        print("padded_image shape: ",padded_image.shape)

    # Stack padded images into a single NumPy array
    sr_padded_array = np.array(padded_images)

    return sr_padded_array, tif_county_id, tif_county_year


In [15]:
nd_sr_arr1, nd_tif_county_id1, nd_tif_county_year1 = process_tif_images(nd_imgs[:100], dir_path)

file_path: SR_NorthDakota_County_37_Year_2020.tif
Image cropped
cropped_image shape:  (7, 328, 3338)
Image Appended to cropped_images
county_id =  37
year =  2020
dimensions:  328 3338
file_path: SR_NorthDakota_County_93_Year_2020.tif
Image cropped
cropped_image shape:  (7, 322, 3338)
Image Appended to cropped_images
county_id =  93
year =  2020
dimensions:  328 3338
file_path: SR_NorthDakota_County_45_Year_2020.tif
Image cropped
cropped_image shape:  (7, 166, 3338)
Image Appended to cropped_images
county_id =  45
year =  2020
dimensions:  328 3338
file_path: SR_NorthDakota_County_69_Year_2020.tif
Image cropped
cropped_image shape:  (7, 322, 3338)
Image Appended to cropped_images
county_id =  69
year =  2020
dimensions:  328 3338
file_path: SR_NorthDakota_County_67_Year_2020.tif
Image cropped
cropped_image shape:  (7, 210, 3338)
Image Appended to cropped_images
county_id =  67
year =  2020
dimensions:  328 3338
file_path: SR_NorthDakota_County_91_Year_2020.tif
Image cropped
cropped_ima

In [16]:
nd_sr_arr2, nd_tif_county_id2, nd_tif_county_year2 = process_tif_images(nd_imgs[100:200], dir_path)

file_path: SR_NorthDakota_County_85_Year_2018.tif
Image cropped
cropped_image shape:  (7, 226, 3338)
Image Appended to cropped_images
county_id =  85
year =  2018
dimensions:  226 3338
file_path: SR_NorthDakota_County_55_Year_2018.tif
Image cropped
cropped_image shape:  (7, 319, 3338)
Image Appended to cropped_images
county_id =  55
year =  2018
dimensions:  319 3338
file_path: SR_NorthDakota_County_97_Year_2019.tif
Image cropped
cropped_image shape:  (7, 205, 3338)
Image Appended to cropped_images
county_id =  97
year =  2019
dimensions:  319 3338
file_path: SR_NorthDakota_County_77_Year_2018.tif
Image cropped
cropped_image shape:  (7, 317, 3338)
Image Appended to cropped_images
county_id =  77
year =  2018
dimensions:  319 3338
file_path: SR_NorthDakota_County_67_Year_2018.tif
Image cropped
cropped_image shape:  (7, 210, 3338)
Image Appended to cropped_images
county_id =  67
year =  2018
dimensions:  319 3338
file_path: SR_NorthDakota_County_97_Year_2018.tif
Image cropped
cropped_ima

In [17]:
nd_sr_arr3, nd_tif_county_id3, nd_tif_county_year3 = process_tif_images(nd_imgs[200:300], dir_path)

file_path: SR_NorthDakota_County_55_Year_2017.tif
Image cropped
cropped_image shape:  (7, 319, 3338)
Image Appended to cropped_images
county_id =  55
year =  2017
dimensions:  319 3338
file_path: SR_NorthDakota_County_69_Year_2017.tif
Image cropped
cropped_image shape:  (7, 322, 3338)
Image Appended to cropped_images
county_id =  69
year =  2017
dimensions:  322 3338
file_path: SR_NorthDakota_County_71_Year_2017.tif
Image cropped
cropped_image shape:  (7, 291, 3338)
Image Appended to cropped_images
county_id =  71
year =  2017
dimensions:  322 3338
file_path: SR_NorthDakota_County_77_Year_2017.tif
Image cropped
cropped_image shape:  (7, 317, 3338)
Image Appended to cropped_images
county_id =  77
year =  2017
dimensions:  322 3338
file_path: SR_NorthDakota_County_67_Year_2016.tif
Image cropped
cropped_image shape:  (7, 210, 3338)
Image Appended to cropped_images
county_id =  67
year =  2016
dimensions:  322 3338
file_path: SR_NorthDakota_County_83_Year_2016.tif
Image cropped
cropped_ima

In [18]:
nd_sr_arr4, nd_tif_county_id4, nd_tif_county_year4 = process_tif_images(nd_imgs[300:400], dir_path)

file_path: SR_NorthDakota_County_55_Year_2015.tif
Image cropped
cropped_image shape:  (7, 319, 3338)
Image Appended to cropped_images
county_id =  55
year =  2015
dimensions:  319 3338
file_path: SR_NorthDakota_County_65_Year_2015.tif
Image cropped
cropped_image shape:  (7, 152, 3338)
Image Appended to cropped_images
county_id =  65
year =  2015
dimensions:  319 3338
file_path: SR_NorthDakota_County_61_Year_2015.tif
Image cropped
cropped_image shape:  (7, 366, 3338)
Image Appended to cropped_images
county_id =  61
year =  2015
dimensions:  366 3338
file_path: SR_NorthDakota_County_15_Year_2015.tif
Image cropped
cropped_image shape:  (7, 320, 3338)
Image Appended to cropped_images
county_id =  15
year =  2015
dimensions:  366 3338
file_path: SR_NorthDakota_County_19_Year_2015.tif
Image cropped
cropped_image shape:  (7, 209, 3338)
Image Appended to cropped_images
county_id =  19
year =  2015
dimensions:  366 3338
file_path: SR_NorthDakota_County_31_Year_2015.tif
Image cropped
cropped_ima

In [19]:
nd_sr_arr5, nd_tif_county_id5, nd_tif_county_year5 = process_tif_images(nd_imgs[400:], dir_path)

file_path: SR_NorthDakota_County_25_Year_2013.tif
Image cropped
cropped_image shape:  (7, 386, 3338)
Image Appended to cropped_images
county_id =  25
year =  2013
dimensions:  386 3338
file_path: SR_NorthDakota_County_49_Year_2013.tif
Image cropped
cropped_image shape:  (7, 361, 3338)
Image Appended to cropped_images
county_id =  49
year =  2013
dimensions:  386 3338
file_path: SR_NorthDakota_County_41_Year_2013.tif
Image cropped
cropped_image shape:  (7, 200, 3338)
Image Appended to cropped_images
county_id =  41
year =  2013
dimensions:  386 3338
file_path: SR_NorthDakota_County_67_Year_2013.tif
Image cropped
cropped_image shape:  (7, 210, 3338)
Image Appended to cropped_images
county_id =  67
year =  2013
dimensions:  386 3338
file_path: SR_NorthDakota_County_51_Year_2013.tif
Image cropped
cropped_image shape:  (7, 161, 3338)
Image Appended to cropped_images
county_id =  51
year =  2013
dimensions:  386 3338
file_path: SR_NorthDakota_County_81_Year_2013.tif
Image cropped
cropped_ima

In [20]:
nd_sr_arr = np.concatenate((nd_sr_arr1, nd_sr_arr2, nd_sr_arr3, nd_sr_arr4, nd_sr_arr5), axis=0)
nd_tif_county_id = nd_tif_county_id1 + nd_tif_county_id2 + nd_tif_county_id3 + nd_tif_county_id4 + nd_tif_county_id5
nd_tif_county_year = nd_tif_county_year1 + nd_tif_county_year2 + nd_tif_county_year3 + nd_tif_county_year4 + nd_tif_county_year5
nd_sr_arr.shape

(530, 7, 438, 3338)

In [21]:
def get_county_names(county_data, tif_county_id):
  # Create a dictionary mapping county IDs to names
  county_mapping = {county_id: county_name for county_id, county_name in county_data}

  # List of county names corresponding to county IDs
  tif_county_name = [county_mapping[county_id] for county_id in tif_county_id]
  return tif_county_name

In [22]:
# Define the list of county IDs and names for ND
county_data_nd = [
    (1, 'Adams'),
    (3, 'Barnes'),
    (5, 'Benson'),
    (7, 'Billings'),
    (9, 'Bottineau'),
    (11, 'Bowman'),
    (13, 'Burke'),
    (15, 'Burleigh'),
    (17, 'Cass'),
    (19, 'Cavalier'),
    (21, 'Dickey'),
    (23, 'Divide'),
    (25, 'Dunn'),
    (27, 'Eddy'),
    (29, 'Emmons'),
    (31, 'Foster'),
    (33, 'Golden Valley'),
    (35, 'Grand Forks'),
    (37, 'Grant'),
    (39, 'Griggs'),
    (41, 'Hettinger'),
    (43, 'Kidder'),
    (45, 'LaMoure'),
    (47, 'Logan'),
    (49, 'McHenry'),
    (51, 'McIntosh'),
    (53, 'McKenzie'),
    (55, 'McLean'),
    (57, 'Mercer'),
    (59, 'Morton'),
    (61, 'Mountrail'),
    (63, 'Nelson'),
    (65, 'Oliver'),
    (67, 'Pembina'),
    (69, 'Pierce'),
    (71, 'Ramsey'),
    (73, 'Ransom'),
    (75, 'Renville'),
    (77, 'Richland'),
    (79, 'Rolette'),
    (81, 'Sargent'),
    (83, 'Sheridan'),
    (85, 'Sioux'),
    (87, 'Slope'),
    (89, 'Stark'),
    (91, 'Steele'),
    (93, 'Stutsman'),
    (95, 'Towner'),
    (97, 'Traill'),
    (99, 'Walsh'),
    (101, 'Ward'),
    (103, 'Wells'),
    (105, 'Williams')
]

nd_tif_county_name = get_county_names(county_data_nd, nd_tif_county_id)

# Wheat Data PreProcessing

In [23]:
import pandas as pd
import numpy as np

In [24]:
def wheat_csv_preprocess(df_path):
  wheat_df = pd.read_csv(df_path)
  wheat_df = wheat_df[['Year', 'County', 'Data Item', 'Value']]
  new_df = wheat_df.groupby(['Year', 'County'])['Value'].sum().reset_index()
  new_df = new_df.rename(columns={'Value': 'Yield (BU/acre)'})

  return new_df

In [25]:
wheat_df_path_nd = '/content/drive/MyDrive/Conservatives-Official/SurfaceReflectance/Output/wheat_yield_nd_2011_2020.csv'
wheat_df_nd = wheat_csv_preprocess(wheat_df_path_nd)
wheat_df_nd.head()

,Year,County,Yield (BU/acre)
0,2011,ADAMS,16.6
1,2011,BARNES,34.0
2,2011,BENSON,34.3
3,2011,BILLINGS,26.5
4,2011,BOTTINEAU,33.8


In [26]:
wheat_df_nd['Year'].value_counts()

2020    51
2011    46
2014    43
2018    41
2012    40
2017    39
2016    37
2015    34
2019    31
2013    20
Name: Year, dtype: int64

## Wheat data X NDVI data processing

In [27]:
def sr_wheat_preprocess(tif_arr, tif_county_id,
                          tif_county_name, tif_county_year, wheat_df):
  temp = {
    'Array_Ind': np.arange(len(tif_arr)),
    'Year': tif_county_year,
    'County_ID': tif_county_id,
    'County': tif_county_name,
  }
  temp_df = pd.DataFrame(temp)
  temp_df['County'] = temp_df['County'].str.upper()

  merged_df = pd.merge(temp_df, wheat_df, on=['Year', 'County'], how='inner')

  # Reorder columns as per your requirement
  merged_df = merged_df[['Array_Ind', 'Year', 'County_ID', 'County', 'Yield (BU/acre)']]
  #print(merged_df.shape)
  array_ind = list(merged_df[['Array_Ind']].values)

  total_img = len(array_ind)
  c = nd_sr_arr.shape[1]
  h = tif_arr.shape[2]
  w = tif_arr.shape[3]
  # Preallocate space for the final array
  new_sr_arr = np.empty((total_img+1, c, h, w)) # Adding 1 for the initial array

  # Read and append each image to the final array
  for index, arr_ind in enumerate(array_ind):
    new_sr_arr[index + 1] = tif_arr[arr_ind]

  new_sr_arr = new_sr_arr[1:]

  return new_sr_arr, merged_df

In [28]:
new_sr_arr_nd, merged_df_nd = sr_wheat_preprocess(nd_sr_arr, nd_tif_county_id, nd_tif_county_name, nd_tif_county_year, wheat_df_nd)
new_sr_arr_nd.shape

(366, 7, 438, 3338)

In [29]:
def save_preprocessed(sr_arr, merged_df, filename):
  sr_yield_preprocessed = {"SR_Array": sr_arr,
                    "County_ID": list(merged_df['County_ID']),
                    "County_Name": list(merged_df['County']),\
                    "Year": list(merged_df['Year']),
                    "Yield": list(merged_df['Yield (BU/acre)'])

                    }

  pickle_filename = '/content/drive/MyDrive/Conservatives-Official/SurfaceReflectance/Output/' + filename

  # Save the dictionary as a pickle file
  with open(pickle_filename, 'wb') as f:
    pickle.dump(sr_yield_preprocessed, f)
  print("File Saved")

In [ ]:
save_preprocessed(new_sr_arr_nd, merged_df_nd, 'sr_yield_nd.pkl')

In [ ]:
def open_pickle_file(filename):
    pickle_filename = '/content/drive/MyDrive/Conservatives-Official/SurfaceReflectance/Output/' + filename

    with open(pickle_filename, 'rb') as f:
        data = pickle.load(f)
    print("File Loaded Successfully")
    return data

In [ ]:
sr_yield_nd = open_pickle_file('/content/drive/MyDrive/Conservatives-Official/SurfaceReflectance/Output/sr_yield_nd.pkl')

In [ ]:
print(sr_yield_nd['SR_Array'].shape)